# Scheduler Playground

In [1]:
from astropy.time import Time
from dateutil.parser import parse as parse_date

from panoptes.pocs.scheduler import create_scheduler_from_config
from panoptes.utils.serializers import from_yaml

In [2]:
scheduler = create_scheduler_from_config()

I 06-05 17:54:40.188 scheduler_config: {'check_file': True, 'constraints': [{'name': 'panoptes.pocs.scheduler.constraint.Altitude'}, {'name': 'panoptes.pocs.scheduler.constraint.MoonAvoidance', 'options': {'separation': 15}}, {'name': 'panoptes.pocs.scheduler.constraint.Duration'}], 'fields_file': '/Users/wilfredo/Projects/panoptes/POCS/conf_files/fields/simple.yaml', 'iers_auto': True, 'iers_url': 'https://storage.googleapis.com/panoptes-assets/iers/ser7.dat', 'type': 'panoptes.pocs.scheduler.dispatch'}
I 06-05 17:54:40.190 Getting IERS data at iers_url='https://storage.googleapis.com/panoptes-assets/iers/ser7.dat'
I 06-05 17:54:40.231 Creating fields from path: /Users/wilfredo/Projects/panoptes/POCS/conf_files/fields/simple.yaml
I 06-05 17:54:40.313 Creating PanDB panoptes
I 06-05 17:54:40.324 Setting new observation to None


### TimeWindow Constraint

The `TimeWindow` constraint takes a `start_time`, `end_time` and an optional `weight` (default is `100.0` for the `TimeWindow` constraint; other constraints the default is `1.0`). The final `score` for the constraint will be multiplied by the `weight`. The weighted scores from each constraint are summed and multiplied by the observation `priority`.

Here we add an exoplanet to the scheduler with a specific time window, then test two different times, one inside the window and the other outside.

In [3]:
scheduler_config = from_yaml("""
      field:
        name: HD 189733
        position: 20h00m43.7135s +22d42m39.0645s
      observation:
        priority: 100
        constraints:
          - name: TimeWindow
            options:
              start_time: "2016-08-13 00:00 HST"  # Example start time for M42
              end_time: "2016-08-13 02:00 HST"  # Example end time for M42
              weight: 100  # This is the default but shown here as example
    """)

In [4]:
scheduler.add_observation(scheduler_config)

In [5]:
t0 = Time(parse_date('2016-08-12 23:30:00 HST'))  # Outside window
t1 = Time(parse_date('2016-08-13 00:30:00 HST'))  # During window

#### Test schedule outside TimeWindow

In [6]:
scheduler.get_observation(time=t0)

I 06-05 17:54:40.755 Applying constraints to observations:
I 06-05 17:54:40.755 Blaze Star
I 06-05 17:54:40.759 	Vetoed by Altitude
I 06-05 17:54:40.759 KIC 8462852
I 06-05 17:54:40.762 	Altitude                      Constraint score:       1.00	Total score:       1.00
I 06-05 17:54:40.764 	Moon Avoidance (15.0 deg)     Constraint score:       0.41	Total score:       1.41
I 06-05 17:54:40.825 	Duration above 30.0 deg       Constraint score:       0.75	Total score:       2.16
I 06-05 17:54:40.826 Wasp 77
I 06-05 17:54:40.829 	Vetoed by Altitude
I 06-05 17:54:40.829 M42
I 06-05 17:54:40.832 	Vetoed by Altitude
I 06-05 17:54:40.832 M45
I 06-05 17:54:40.835 	Vetoed by Altitude
I 06-05 17:54:40.835 Beehive Cluster
I 06-05 17:54:40.839 	Vetoed by Altitude
I 06-05 17:54:40.839 HD 189733
I 06-05 17:54:40.842 	Altitude                      Constraint score:       1.00	Total score:       1.00
I 06-05 17:54:40.843 	Moon Avoidance (15.0 deg)     Constraint score:       0.32	Total score:       1.32

('KIC 8462852', np.float64(216.02979602449733))

#### Test schedule inside TimeWindow

In [7]:
scheduler.get_observation(time=t1)

I 06-05 17:54:41.478 Applying constraints to observations:
I 06-05 17:54:41.479 Blaze Star
I 06-05 17:54:41.482 	Vetoed by Altitude
I 06-05 17:54:41.483 KIC 8462852
I 06-05 17:54:41.486 	Altitude                      Constraint score:       1.00	Total score:       1.00
I 06-05 17:54:41.488 	Moon Avoidance (15.0 deg)     Constraint score:       0.41	Total score:       1.41
I 06-05 17:54:41.535 	Duration above 30.0 deg       Constraint score:       0.69	Total score:       2.10
I 06-05 17:54:41.535 Wasp 77
I 06-05 17:54:41.538 	Vetoed by Altitude
I 06-05 17:54:41.539 M42
I 06-05 17:54:41.541 	Vetoed by Altitude
I 06-05 17:54:41.542 M45
I 06-05 17:54:41.545 	Vetoed by Altitude
I 06-05 17:54:41.545 Beehive Cluster
I 06-05 17:54:41.548 	Vetoed by Altitude
I 06-05 17:54:41.549 HD 189733
I 06-05 17:54:41.551 	Altitude                      Constraint score:       1.00	Total score:       1.00
I 06-05 17:54:41.553 	Moon Avoidance (15.0 deg)     Constraint score:       0.32	Total score:       1.32

('HD 189733', np.float64(10195.749063586147))